## Back Test Results Binary Classification

In [ ]:
import pandas as pd
from datetime import datetime, timezone
from pycaret.datasets import get_data
from pycaret.classification import *
import yaml
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    pair = cfg['currency']['pair']
    frac = cfg['model']['frac']
    random_state = cfg['model']['random_state']

In [ ]:
print('Pair:', pair)
print('Fraction:', frac)
print('Random State:', random_state)

In [ ]:
now = datetime.now()
today = now.strftime("%d-%m-%Y_%I-%M_%p")

In [ ]:
dataset = pd.read_csv('01_Back_Test_Data.csv')
dataset.shape

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
dataset['Rec1_Close_Score'] = dataset['Rec1_Close_Score'].round()
dataset['Rec1_High_Score'] = dataset['Rec1_High_Score'].round()
dataset['Rec1_Low_Score'] = dataset['Rec1_Low_Score'].round()

dataset['Rec2_Close_Score'] = dataset['Rec2_Close_Score'].round()
dataset['Rec2_High_Score'] = dataset['Rec2_High_Score'].round()
dataset['Rec2_Low_Score'] = dataset['Rec2_Low_Score'].round()

dataset['Rec3_Close_Score'] = dataset['Rec3_Close_Score'].round()
dataset['Rec3_High_Score'] = dataset['Rec3_High_Score'].round()
dataset['Rec3_Low_Score'] = dataset['Rec3_Low_Score'].round()

dataset['Rec4_Close_Score'] = dataset['Rec4_Close_Score'].round()
dataset['Rec4_High_Score'] = dataset['Rec4_High_Score'].round()
dataset['Rec4_Low_Score'] = dataset['Rec4_Low_Score'].round()

In [ ]:
dataset['Rec1_HH'] = abs(dataset['Rec1_HH'])
dataset['Rec1_LL'] = abs(dataset['Rec1_LL'])
dataset['Rec2_HH'] = abs(dataset['Rec2_HH'])
dataset['Rec2_LL'] = abs(dataset['Rec2_LL'])
dataset['Rec3_HH'] = abs(dataset['Rec3_HH'])
dataset['Rec3_LL'] = abs(dataset['Rec3_LL'])
dataset['Rec4_HH'] = abs(dataset['Rec4_HH'])
dataset['Rec4_LL'] = abs(dataset['Rec4_LL'])

In [ ]:
dataset.head(10)

## Drop Predictions with Less than n% LR Fit

In [ ]:
dataset = dataset[dataset['Current_Market_Fit'] > 16]
dataset = dataset.reset_index()
del dataset['index']

In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset = dataset.drop([
    'Candle_No',
    'Current_Market_Fit',
#    'Rec1_Close_Score',
    'Rec1_High_Score',
    'Rec1_Low_Score',
    'Rec1_HH',
    'Rec1_LL',
#    'Rec2_Close_Score',
    'Rec2_High_Score',
    'Rec2_Low_Score',
    'Rec2_HH',
    'Rec2_LL',
#    'Rec3_Close_Score',
    'Rec3_High_Score',
    'Rec3_Low_Score',
    'Rec3_HH',
    'Rec3_LL',
#    'Rec4_Close_Score',
    'Rec4_High_Score',
    'Rec4_Low_Score',
    'Rec4_HH',
    'Rec4_LL',
], axis=1)

In [ ]:
dataset.head(20)

In [ ]:
data = dataset.sample(frac=frac, random_state=random_state).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))
print('Fraction: ' + str(frac))
#print('Random State: ', str(random_state))

# Create Unseen DataFrame

In [ ]:
data_unseen.shape

In [ ]:
data_unseen.head()

In [ ]:
exp_name = setup(data = data,
                 target = 'Current_Market',
#                 fold_shuffle=True,
                 session_id=123,
                 silent=True,
#                 n_jobs = 64,
#                 imputation_type='iterative'
                ) 

In [ ]:
compare_models()

In [ ]:
break

In [ ]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    model = cfg['model']['name']
    
print('Model Name:', model)

In [ ]:
MODEL = create_model(model)

In [ ]:
plot_model(MODEL, plot = 'confusion_matrix')

In [ ]:
TUNED_MODEL = tune_model(MODEL)

In [ ]:
plot_model(TUNED_MODEL, plot = 'auc')

In [ ]:
plot_model(TUNED_MODEL, plot = 'pr')

In [ ]:
plot_model(TUNED_MODEL, plot='feature')

In [ ]:
plot_model(TUNED_MODEL, plot = 'confusion_matrix')

In [ ]:
predict_model(TUNED_MODEL);

In [ ]:
FINAL_MODEL = finalize_model(TUNED_MODEL)

In [ ]:
print(FINAL_MODEL)

In [ ]:
predict_model(FINAL_MODEL);

In [ ]:
unseen_predictions = predict_model(FINAL_MODEL, data=data_unseen)
unseen_predictions.head()

In [ ]:
plot_model(FINAL_MODEL, plot = 'auc')

In [ ]:
pkl_name = (today + "_" + model + "_" + pair)

In [ ]:
save_model(FINAL_MODEL, pkl_name)

In [ ]:
SAVED_FINAL_MODEL = load_model(pkl_name)

In [ ]:
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)

In [ ]:
data_unseen.shape

In [ ]:
new_prediction.head(20)

In [ ]:
file = open(today + "_" + model + "_binary_classification_log_" + pair + ".txt", "w")
file.write ("Date: " + today + "\n" + \
            "Currency Pair: " + pair + "\n" + \
            "Model: " + model + "\n" + \
            "Frac: " + str(frac) + "\n" + \
            "Random State: " + str(random_state) + "\n" + \
            "Model Input: " + str(dataset.columns) + "\n"  + \
            "Training Data Size: " + str(data.shape) + "\n" + \
            "Test Data Size: " + str(data_unseen.shape) + "\n"
           )
file.close()